In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy import sparse
from collections import Counter
from sklearn.preprocessing import MinMaxScaler

from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
df = pd.read_csv('../data\games.csv')
df.columns

Index(['AppID', 'Name', 'Release date', 'Estimated owners', 'Peak CCU',
       'Required age', 'Price', 'DLC count', 'About the game',
       'Supported languages', 'Full audio languages', 'Reviews',
       'Header image', 'Website', 'Support url', 'Support email', 'Windows',
       'Mac', 'Linux', 'Metacritic score', 'Metacritic url', 'User score',
       'Positive', 'Negative', 'Score rank', 'Achievements', 'Recommendations',
       'Notes', 'Average playtime forever', 'Average playtime two weeks',
       'Median playtime forever', 'Median playtime two weeks', 'Developers',
       'Publishers', 'Categories', 'Genres', 'Tags', 'Screenshots', 'Movies'],
      dtype='object')

In [3]:
df.head()

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,Galactic Bowling is an exaggerated and stylize...,['English'],...,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
1,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
2,1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,0,4.99,0,Jolt Project: The army now has a new robotics ...,"['English', 'Portuguese - Brazil']",...,0,0,0,Campião Games,Campião Games,Single-player,"Action,Adventure,Indie,Strategy",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
3,1355720,Henosis™,"Jul 23, 2020",0 - 20000,0,0,5.99,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Odd Critter Games,Odd Critter Games,"Single-player,Full controller support","Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
4,1139950,Two Weeks in Painland,"Feb 3, 2020",0 - 20000,0,0,0.00,0,ABOUT THE GAME Play as a hacker who has arrang...,"['English', 'Spanish - Spain']",...,0,0,0,Unusual Games,Unusual Games,"Single-player,Steam Achievements","Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


In [4]:
#df.date.unique()
df['Release date'] = pd.to_datetime(df['Release date'], format='%b %d, %Y', errors='coerce')
select_cond = df['Peak CCU'] > 0
df = df[select_cond]
# df_recency = df.sort_values(by='Release date')
# df.head()

In [5]:
df.isna().sum()

AppID                             0
Name                              0
Release date                     47
Estimated owners                  0
Peak CCU                          0
Required age                      0
Price                             0
DLC count                         0
About the game                   24
Supported languages               0
Full audio languages              0
Reviews                       16461
Header image                      0
Website                        8047
Support url                    8325
Support email                  3947
Windows                           0
Mac                               0
Linux                             0
Metacritic score                  0
Metacritic url                17946
User score                        0
Positive                          0
Negative                          0
Score rank                    20908
Achievements                      0
Recommendations                   0
Notes                       

In [6]:
df.shape

(20926, 39)

In [7]:
percent = .90
column_percentages = df.count() / len(df)

df = df.dropna(axis=1, thresh=len(df) * percent)
df.shape

(20926, 32)

In [8]:
df.isna().sum()

AppID                            0
Name                             0
Release date                    47
Estimated owners                 0
Peak CCU                         0
Required age                     0
Price                            0
DLC count                        0
About the game                  24
Supported languages              0
Full audio languages             0
Header image                     0
Windows                          0
Mac                              0
Linux                            0
Metacritic score                 0
User score                       0
Positive                         0
Negative                         0
Achievements                     0
Recommendations                  0
Average playtime forever         0
Average playtime two weeks       0
Median playtime forever          0
Median playtime two weeks        0
Developers                       0
Publishers                      68
Categories                     305
Genres              

In [9]:
df.dropna(inplace=True)
df.shape

(17728, 32)

In [10]:
column_object = df.dtypes[df.dtypes == 'object'].keys()

In [11]:
multi_label_columns = ['Genres','Supported languages','Full audio languages','Categories']
column_object = column_object.drop(multi_label_columns)
column_object

Index(['Name', 'Estimated owners', 'About the game', 'Header image',
       'Developers', 'Publishers', 'Tags', 'Screenshots', 'Movies'],
      dtype='object')

In [12]:
li = list(df.dtypes[df.dtypes == 'object'].keys())
li

['Name',
 'Estimated owners',
 'About the game',
 'Supported languages',
 'Full audio languages',
 'Header image',
 'Developers',
 'Publishers',
 'Categories',
 'Genres',
 'Tags',
 'Screenshots',
 'Movies']

In [13]:
# df['Categories'] = df['Categories'].str.split(',')
# df['Genres'] = df['Genres'].str.split(',')
df.isna().sum()

AppID                         0
Name                          0
Release date                  0
Estimated owners              0
Peak CCU                      0
Required age                  0
Price                         0
DLC count                     0
About the game                0
Supported languages           0
Full audio languages          0
Header image                  0
Windows                       0
Mac                           0
Linux                         0
Metacritic score              0
User score                    0
Positive                      0
Negative                      0
Achievements                  0
Recommendations               0
Average playtime forever      0
Average playtime two weeks    0
Median playtime forever       0
Median playtime two weeks     0
Developers                    0
Publishers                    0
Categories                    0
Genres                        0
Tags                          0
Screenshots                   0
Movies  

In [14]:
tf = TfidfVectorizer()
text_data_about = df['About the game'].astype(str)
text_data_genre = df['Genres'].astype(str)
text_data_categories = df['Categories'].astype(str)
text_data_developers = df['Developers'].astype(str)
text_data_title = df['Name'].astype(str)
text_data = text_data_about + ' ' + text_data_genre + ' ' + text_data_developers + ' ' +  text_data_categories + ' ' + text_data_title
tfidf_matrix = tf.fit_transform(text_data)
similarity_matrix = cosine_similarity(tfidf_matrix,tfidf_matrix)


In [15]:
def CosineGameRecommended(gamename:str, tfidf_vectorizer, similarity_matrix, df, recommended_games:int=5):
    # Reset the index to ensure it is continuous
    df_reset = df.reset_index(drop=True)

    # Combine text data from 'About the game' and 'Genres'
    text_data_combined = df_reset['About the game'].astype(str) + ' ' + df_reset['Genres'].astype(str) + ' ' + df_reset['Categories'].astype(str)  + ' ' + df_reset['Developers'].astype(str)  + ' ' + df_reset['Name'].astype(str) 

    # Transform the combined text data into a TF-IDF vector
    game_tfidf_vector = tfidf_vectorizer.transform([text_data_combined[df_reset['Name'] == gamename].values[0]])

    # Calculate cosine similarity for the given game vector
    similarity_scores = cosine_similarity(game_tfidf_vector, tfidf_matrix).flatten()

    # Get the indices of games with the highest similarity scores
    top_indices = np.argsort(similarity_scores)[-recommended_games-1:-1][::-1]

    # Retrieve corresponding games using the reset index
    top_games = df_reset.loc[top_indices, 'Name'].tolist()

    return pd.DataFrame(top_games)

In [16]:
recommendations = CosineGameRecommended('Call of Duty', tf, similarity_matrix, df)
print("Top Recommendations:", recommendations)

Top Recommendations:                             0
0  Assetto Corsa Competizione
1    GTR 2 FIA GT Racing Game
2             LOTUS-Simulator
3       Classic Sport Driving
4  Victory: The Age of Racing


In [ ]:

# Model initiation
model = NearestNeighbors(metric='euclidean')

# Fit model to the data
model.fit(df)

NearestNeighbors(metric='euclidean')

In [ ]:

def getRecommended(gamename: str):
    distances, neighbors = model.kneighbors(df.loc[gamename].values.reshape(1, -1), n_neighbors=5)
    
    similar_games = []
    for index in neighbors[0][1:]:
        similar_games.append(df.index[index])
    
    similar_distance = [f"{distance}%" for distance in distances[0][1:]]
    
    return pd.DataFrame(data={"Game": similar_games, "Similarity": similar_distance})



In [ ]:

getRecommended('New World')


c:\Users\dsouz\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,Game,Similarity
0,No Man's Sky,31709.169115572866%
1,Lost Ark,42741.10588414974%
2,Sid Meier’s Civilization® VI,48868.443058890305%
3,VRChat,51675.956132422936%


In [ ]:
cosine_sim = cosine_similarity(df)
cosine_sim_df = pd.DataFrame(cosine_sim, index=df.index)
cosine_sim_df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,17718,17719,17720,17721,17722,17723,17724,17725,17726,17727
Name,,,,,,,,,,,,,,,,,,,,,
Wartune Reborn,1.000000,0.326435,0.845399,0.977848,0.886340,0.897066,0.961432,0.968365,0.931844,0.992783,...,0.132336,0.195325,0.669748,0.828459,0.704412,0.274610,0.252481,0.016647,0.822330,0.166967
TD Worlds,0.326435,1.000000,0.615432,0.384974,0.339470,0.341706,0.313191,0.334256,0.328448,0.407450,...,0.907618,0.215242,0.348604,0.352642,0.346576,0.238196,0.970544,0.170921,0.614116,0.978376
MazM: Jekyll and Hyde,0.845399,0.615432,1.000000,0.915607,0.947803,0.938540,0.729938,0.918775,0.931952,0.846235,...,0.418654,0.300140,0.830469,0.920936,0.849246,0.438812,0.565411,0.195937,0.987263,0.471990


In [ ]:


def CosineGameRecommended(gamename:str, recommended_games:int=5):
  
  arr, ind = np.unique(cosine_sim_df.loc[gamename], return_index=True)

  similar_games = []
  for index in ind[-(recommended_games+1):-1]:
    similar_games.append(df.index[index])

  cosine_score = []
  for score in arr[-(recommended_games+1):-1]:
    cosine_score.append(score)

  return pd.DataFrame(data = {"Game" : similar_games, "Cosine Similarity" : cosine_score}).sort_values(by='Cosine Similarity')

In [ ]:
CosineGameRecommended('New World')

,Game,Cosine Similarity
0,Generation Zero®,0.999989
1,Evolve Stage 2,0.999993
2,Propnight,0.999996
3,Digimon Masters Online,0.999999
4,Warface,0.999999


In [ ]:
getRecommended('New World')

c:\Users\dsouz\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,Game,Similarity
0,No Man's Sky,31709.169115572866%
1,Lost Ark,42741.10588414974%
2,Sid Meier’s Civilization® VI,48868.443058890305%
3,VRChat,51675.956132422936%
